# nn
> Neural Net based on `Value` nodes

In [ ]:
#| default_exp nn

In [ ]:
#| hide
from micrograd_nbdev.engine import Value
import random
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
class Module:
    "Base class"
    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0
    
    def parameters(self):
        return []

In [ ]:
#| export
class Neuron(Module):
    # Neuron is a single calculation node
    def __init__(self,
                 nin, # number of inputs (parameters) for each node
                 nonlin=True # whether to use nonlinearity
                ):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin
    
    def __call__(self, x):
        return sum([xi*wi for (xi, wi) in zip(x, self.w)]) + self.b
    
    def parameters(self):
        return [weight for weight in self.w] + [self.b]
    
    def __repr__(self):
        return f"{'ReLU' if self.nonlin  else  'Linear'} Neuron({len(self.w)})"

In [ ]:
n = Neuron(5)
n

ReLU Neuron(5)

In [ ]:
n([Value(1),Value(1),Value(1),Value(1),Value(1)])

Value(data=0.6715391519237188)

In [ ]:
class Layer(Module):
    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]
    
    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        # if nout is 1 then simple return an item rather than a list on one item
        return out[0] if len(out) == 1 else out
        
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]
    
    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"


In [ ]:
l = Layer(2, 1)

In [ ]:
l.parameters()

[Value(data=-0.5959432833313711),
 Value(data=-0.8379611292744993),
 Value(data=0)]

In [ ]:
l([Value(2.0), Value(1.0)])

[Value(data=-2.0298476959372413)]